# 开源方案
- https://github.com/yunsuxiaozi/AI-and-competition/blob/main/2024kddWhoiswhotop37solution/2024kdd-data.ipynb

In [2]:
#necessary
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
import json#用于读取和写入json数据格式
import re#用于正则表达式提取
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

## 文本可读性指数

In [ ]:
#文本的自动可读性指数 旨在衡量文本的可理解性.输出是理解课文所需的美国年级水平的近似表示.
#https://www.nhooo.com/note/qa0tpe.html
#初步理解:相同词数的情况下,句子越少,说明句子相对来说会很长,越长越不容易理解.words/sentence就会越大.
#字符数相同的情况下,词数越多,单词越短,短的单词可能简单,所以就好理解.characters/words变小.
#数值小就好理解,数值大就不好理解.具体的公式可能用数据做过实验得出?
def ARI(txt):
    if txt==None:
        txt="q"
    characters=len(txt)
    words=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))#空格,换行符,句号,问号,感叹号,逗号分开.
    sentence=len(re.split('\\.|\\?|\\!',txt))#句号,问号,感叹号分开的句子.
    ari_score=4.71*(characters/words)+0.5*(words/sentence)-21.43
    return ari_score
"""
http://www.supermagnus.com/mac/Word_Counter/index.html
McAlpine EFLAW© Test
     (W + SW) / S
McAlpine EFLAW© Readability
     Scale:
     1-20: Easy
     21-25: Quite Easy
     26-29: Mildly Difficult
     ≥ 30: Very Confusing
     S:total sentences
     W:total words
"""
def McAlpine_EFLAW(txt):
    if txt==None:
        txt="q"
    W=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))#空格,换行符,句号,问号,感叹号,逗号分开.
    S=len(re.split('\\.|\\?|\\!',txt))#句号,问号,感叹号分开的句子.
    mcalpine_eflaw_score=(W+S*W)/S
    return mcalpine_eflaw_score
"""
https://readable.com/readability/coleman-liau-readability-index/

=0.0588*L-0.296*S-15.8
L是每100个单词有多少个字母,S是平均每100个单词有多少句子.
"""
def CLRI(txt):
    if txt==None:
        txt="q"
    characters=len(txt)
    words=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))#空格,换行符,句号,问号,感叹号,逗号分开.
    sentence=len(re.split('\\.|\\?|\\!',txt))#句号,问号,感叹号分开的句子.
    L=100*characters/words
    S=100*sentence/words
    clri_score=0.0588*L-0.296*S-15.8
    return clri_score

#统计txt文本中出现find_word几次
def calwordcnt(txt,find_word):
    wordcnt=0
    words=txt.split()
    for word in words:
        word=word.lower()
        wordcnt+=int(word==find_word)
    return wordcnt

## 右侧长尾或者右偏态数据处理

In [ ]:
train_feats.replace([np.inf, -np.inf], np.nan, inplace=True)
valid_feats.replace([np.inf, -np.inf], np.nan, inplace=True)

for col in valid_feats.columns:
    #数据呈现右偏,并且可以进行log1p处理
    if (train_feats[col].skew()>1) and train_feats[col].min()>-1:
        print(f"skew:{col}")
        train_feats[col]=np.log1p(train_feats[col])
        valid_feats[col]=np.log1p(valid_feats[col])

## 左侧长尾或者右偏态数据

- 反向对数变换（Inverse Log Transformation）：
    - 对于左偏分布的数据，可以进行反向对数变换。即对数据取负值，然后再应用对数变换。

In [ ]:
'''
先取负值（-train_feats[col]）将左偏数据转换为右偏数据。
减去最小值以确保所有值为正（避免log(0)的情况）。
再加1确保所有值大于0，然后应用log1p变换。
'''

train_feats[col] = np.log1p(-train_feats[col] - train_feats[col].min() + 1)
valid_feats[col] = np.log1p(-valid_feats[col] - valid_feats[col].min() + 1)

- 平方变换（Square Transformation）：
    - 另一种方法是对数据进行平方变换。这对于数据偏度不严重的情况效果较好。
    
- 平方根变换（Square Root Transformation）：
    - 对于左偏数据，可以取平方根来使数据分布更加对称。

In [4]:
'''
对数据进行平方变换，减少偏度。
'''
train_feats[col] = train_feats[col] ** 2
valid_feats[col] = valid_feats[col] ** 2


'''
平方根变换
'''

train_feats[col] = np.sqrt(train_feats[col] - train_feats[col].min() + 1)
valid_feats[col] = np.sqrt(valid_feats[col] - valid_feats[col].min() + 1)